In [1]:
def nmr_file_parser(peak_file_dir):
    file = open(peak_file_dir, 'r')
    c1,c2=[],[]
    h1,h2=[],[]
    cross,reflected=[],[]
    d1,d2=[],[]
    lab=[]
    for line in file:
        data = line.split()
        res1=residue_parser(data[0])
        res2=residue_parser(data[1])
        lab.append(res1+','+res2)
        c1.append(data[2])
        c2.append(data[4])
        h1.append(data[3])
        h2.append(data[5])
        cross.append(data[6])
        reflected.append(data[7])
        d1.append(data[14])
        d2.append(data[15])
    
    return lab,c1,h2,c2,h2,cross,reflected,d1,d2
    
        
def residue_parser(residue_str):
    res_letter = residue_str[0]
    integers = [int(s) for s in re.findall(r'-?\d+\.?\d*', residue_str)]
    out_str = ''
    if(res_letter=='I'):
        out_str='ILE'+' '+str(integers[0])+' A'
    elif(res_letter=='A'):
        out_str='ALA'+' '+str(integers[0])+' A'
    elif(res_letter=='L'):
        if('D1' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' A'
        elif('D2' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' B'
    elif(res_letter=='V'):
        if('G1' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' A'
        elif('G2' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' B'
    if(out_str==''):
        print 'BLANK AMINO ACID CODE. AMINO ACID STRING TO BE PARSED = ',residue_str
        sys.exit(100)
    return out_str


peak_file_dir='input/correlate.3'
lab,c1,h2,c2,h2,cross,reflected,d1,d2=nmr_file_parser(peak_file_dir)



IOError: [Errno 2] No such file or directory: 'input/correlate.3'

In [92]:
def nmr_masker(h1,h2,c1,c2,threshold=0.01):
    #Takes in 4 2D square matrices. Will then calculate the difference between peaks, in an expanded 4D matrix. 
    #If two peaks are too close, IE closer than the threshold level, we eliminate them from the data set
    #IE we mask them from the 2D square matrix, setting the correspond indices to True in the mask
    
    #SETUP MATRICES TO STORE DATA:
    num=h1.shape[0]
    mask=numpy.zeros((num,num),dtype=bool)
    diff_matrix=numpy.zeros((num,num,num,num))
    
    #SETUP INDEXING MATRICES:
    i,j,k,l=numpy.arange(num),numpy.arange(num),numpy.arange(num),numpy.arange(num)
    ii,jj,kk,ll=numpy.meshgrid(i,j,k,l)
    
    #EXPAND TO A 4D DIFFERENCE MATRIX - TAKES DIFFERENCE BETWEEN PEAK (ii,jj) and peak (kk,ll):
    diff_matrix[ii,jj,kk,ll]=2.*numpy.sqrt(((h1[ii,jj]-h1[kk,ll])/(h1[ii,jj]+h1[kk,ll]))**2 + ((h2[ii,jj]-h2[kk,ll])/(h2[ii,jj]+h2[kk,ll]))**2 + ((c1[ii,jj]-c1[kk,ll])/(c1[ii,jj]+c1[kk,ll]))**2 + ((c2[ii,jj]-c2[kk,ll])/(c2[ii,jj]+c2[kk,ll]))**2)
    
    #CUT-OFF AT WHERE THIS DIFFERENCE IS SMALLER THAN THRESHOLD:
    sum_matrix=diff_matrix<threshold
    
    #FIND "COLUMNS" ALONG WHICH THIS SUM IS GREATER THAN EXPECTED 1
    sum_matrix=numpy.sum(sum_matrix,axis=(2,3))
    mask=sum_matrix<2
    return mask

h1,h2=numpy.array([[1.05,1.23],[1.07,1.59]]),numpy.array([[1.05,1.23],[1.05,1.59]])
c1,c2=numpy.array([[10.59,8.39],[10.59,12.31]]),numpy.array([[9.37,8.39],[9.37,12.31]])
mask=nmr_masker(h1,h2,c1,c2)
print 'RAW DATA = \n',h1
print 'MASKED DATA = \n',h1[mask]


RAW DATA = 
[[1.05 1.23]
 [1.07 1.59]]
MASKED DATA = 
[1.05 1.23 1.07 1.59]


In [54]:
def d1_masker(data,threshold=0.01):
    num=data.shape[0]
    i,j=numpy.arange(num),numpy.arange(num)
    ii,jj=numpy.meshgrid(i,j,indexing='ij')
    diff_matrix=numpy.zeros((num,num))
    diff_matrix[ii,jj]=numpy.sqrt(((data[ii]-data[jj])/(data[ii]+data[jj]))**2)
    print 'diff_matrix =\n',diff_matrix
    bool_matrix=diff_matrix<threshold
    print 'bool_matrix = \n',bool_matrix
    sum_arr=numpy.sum(bool_matrix,axis=0)-1
    print 'sum_arr = \n',sum_arr
    bool_arr=sum_arr<1
    print 'bool_arr = \n',bool_arr
    return bool_arr
    
data=numpy.array([1.,1.05,1.03,1.04,1.25,1.59,1.83,2.01])
mask=d1_masker(data)
print 'RAW DATA = \n',data
print 'MASKED DATA = \n',data[mask]

diff_matrix =
[[0.         0.02439024 0.01477833 0.01960784 0.11111111 0.22779923
  0.29328622 0.33554817]
 [0.02439024 0.         0.00961538 0.00478469 0.08695652 0.20454545
  0.27083333 0.31372549]
 [0.01477833 0.00961538 0.         0.00483092 0.09649123 0.21374046
  0.27972028 0.32236842]
 [0.01960784 0.00478469 0.00483092 0.         0.09170306 0.20912548
  0.27526132 0.31803279]
 [0.11111111 0.08695652 0.09649123 0.09170306 0.         0.11971831
  0.18831169 0.23312883]
 [0.22779923 0.20454545 0.21374046 0.20912548 0.11971831 0.
  0.07017544 0.11666667]
 [0.29328622 0.27083333 0.27972028 0.27526132 0.18831169 0.07017544
  0.         0.046875  ]
 [0.33554817 0.31372549 0.32236842 0.31803279 0.23312883 0.11666667
  0.046875   0.        ]]
bool_matrix = 
[[ True False False False False False False False]
 [False  True  True  True False False False False]
 [False  True  True  True False False False False]
 [False  True  True  True False False False False]
 [False False False False  Tru

In [93]:
def nmr_masker(h1,h2,c1,c2,threshold=0.01):
    #Takes in 4 2D square matrices. Will then calculate the difference between peaks, in an expanded 4D matrix. 
    #If two peaks are too close, IE closer than the threshold level, we eliminate them from the data set
    #IE we mask them from the 2D square matrix, setting the correspond indices to True in the mask
    
    #SETUP MATRICES TO STORE DATA:
    num=h1.shape[0]
    mask=numpy.zeros((num,num),dtype=bool)
    diff_matrix=numpy.zeros((num,num,num,num))
    
    #SETUP INDEXING MATRICES:
    i,j,k,l=numpy.arange(num),numpy.arange(num),numpy.arange(num),numpy.arange(num)
    ii,jj,kk,ll=numpy.meshgrid(i,j,k,l)
    
    #EXPAND TO A 4D DIFFERENCE MATRIX - TAKES DIFFERENCE BETWEEN PEAK (ii,jj) and peak (kk,ll):
    diff_matrix[ii,jj,kk,ll]=2.*numpy.sqrt(((h1[ii,jj]-h1[kk,ll])/(h1[ii,jj]+h1[kk,ll]))**2 + ((h2[ii,jj]-h2[kk,ll])/(h2[ii,jj]+h2[kk,ll]))**2 + ((c1[ii,jj]-c1[kk,ll])/(c1[ii,jj]+c1[kk,ll]))**2 + ((c2[ii,jj]-c2[kk,ll])/(c2[ii,jj]+c2[kk,ll]))**2)
    
    #CUT-OFF AT WHERE THIS DIFFERENCE IS SMALLER THAN THRESHOLD:
    sum_matrix=diff_matrix<threshold
    
    #FIND "COLUMNS" ALONG WHICH THIS SUM IS GREATER THAN EXPECTED 1
    sum_matrix=numpy.sum(sum_matrix,axis=(2,3))
    mask=sum_matrix<2
    return mask

def peak_file_parser(peak_file_dir):
    file = open(peak_file_dir, 'r')
    cross,reflected={},{}
    diag_1,diag_2={},{}
    for line in file:
        data = line.split()
        res1 = residue_parser(data[0])
        res2 = residue_parser(data[1])
        v1,v2=data[6],data[7]
        a1,a2=data[14],data[15]
        if not(res1 in cross):
            cross[res1]={}
            reflected[res1]={}
            diag_2[res1]={}
            diag_1[res1]={}
        cross[res1][res2]=v1
        reflected[res1][res2]=v2
        diag_1[res1][res2]=a1
        diag_2[res1][res2]=a2
    map_key,map_index={},{}
    i=0
    for res1 in cross:
        map_key[res1]=i
        map_index[i]=res1
        i+=1
    
    num = len(cross)
    cross_matrix, reflected_matrix, d1_matrix, d2_matrix = numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num))
    mask_matrix=numpy.ones((num,num),dtype=bool)
    for res1 in cross:
        for res2 in cross[res1]:
            i1,i2=map_key[res1],map_key[res2]
            cross_matrix[i1,i2]=cross[res1][res2]
            reflected_matrix[i1,i2]=reflected[res1][res2]
            d1_matrix[i1,i2]=diag_1[res1][res2]
            d2_matrix[i1,i2]=diag_2[res1][res2]
            mask_matrix[i1,i2]=False
    
    return cross_matrix,reflected_matrix,d1_matrix,d2_matrix,map_key,map_index,mask_matrix
    
        
def residue_parser(residue_str):
    res_letter = residue_str[0]
    integers = [int(s) for s in re.findall(r'-?\d+\.?\d*', residue_str)]
    out_str = ''
    if(res_letter=='I'):
        out_str='ILE'+' '+str(integers[0])+' A'
    elif(res_letter=='A'):
        out_str='ALA'+' '+str(integers[0])+' A'
    elif(res_letter=='L'):
        if('D1' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' A'
        elif('D2' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' B'
    elif(res_letter=='V'):
        if('G1' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' A'
        elif('G2' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' B'
    if(out_str==''):
        print 'BLANK AMINO ACID CODE. AMINO ACID STRING TO BE PARSED = ',residue_str
        sys.exit(100)
    return out_str


cross,reflected,d1,d2,map_key,map_index,mask_matrix=peak_file_parser('input/correlate.3')



IOError: [Errno 2] No such file or directory: 'input/correlate.3'

In [94]:
def get_pdb_coord_methyl(pdb_file):
    #Initialize stuff - Set which file to open; initilaize lists for all atoms in file, and then result list - methyl_list
    print pdb_file
    file = open(pdb_file, 'r')
    atom_list = []
    methyl_list = {}
    
    #Iterate over lines in file, and fill atom_list with position, type & residue (type & number) for each atom:
    for line in file:
        if(line[0:4]=='ATOM'):
#             print line
            atom_number = line[6:12]
            atom_name = line[13:16]
            residue_name = line[17:20]
            residue_number = int(line[22:26].strip())
            x_coord = float(line[30:38])
            y_coord = float(line[38:46])
            z_coord = float(line[46:54])
            
            atom = {}
            atom['num']  = atom_number
            atom['ele'] = atom_name
            atom['residue'] = residue_name + ' ' + str(residue_number)
            atom['x'], atom['y'], atom['z'] = x_coord, y_coord, z_coord
            atom_list.append(atom)

#                 print 'ATOM = ', atom_name
#                 print 'RESIDUE = ', residue_name
#                 print 'X = ', x_coord
#                 print 'Y = ', y_coord
#                 print 'Z = ', z_coord
#                 print line
    
    #Now iterate over all atoms in atom_list and group in residue (type & number):
    residue_list = {}
    for atom in atom_list:
        if(atom['residue'] in residue_list):
            residue_list[atom['residue']][atom['ele']]=(atom['x'],atom['y'],atom['z'])
        else:
            residue_list[atom['residue']]={}
            residue_list[atom['residue']][atom['ele']]=(atom['x'],atom['y'],atom['z'])

    #Now iterate over residues and generate methyls from ILE, LEU, VAL:
    #General strategy - Each methyl consits of 3 vectors - c_m, c_a, c_b. These totally specify a methyl groups position & orientation, *including* psi angle
    #REF:
#     def CalcMetGeom(c_m, c_a, c_b, r_m=1.09, theta_m=numpy.arccos(1./3.)):
    residues = []
    count_ile, count_leu, count_val,count_ala = 0,0,0,0
    for key in residue_list:
        residue = residue_list[key]
        if(key[0:3]=='ILE'):
            residues.append(key)
            methyl = CalcMetGeom(residue['CD1'], residue['CG1'], residue['CB '])
            methyl_list[key+' A'] = methyl
            count_ile+=1
        elif(key[0:3]=='LEU'):
            residues.append(key)
            methyl = CalcMetGeom(residue['CD1'], residue['CG '], residue['CB '])
            methyl_list[key + ' A'] = methyl
            methyl = CalcMetGeom(residue['CD2'], residue['CG '], residue['CB '])
            methyl_list[key + ' B'] = methyl
            count_leu+=1
        elif(key[0:3]=='VAL'):
            residues.append(key)
            methyl = CalcMetGeom(residue['CG1'], residue['CB '], residue['CA '])
            methyl_list[key + ' A'] = methyl
            methyl = CalcMetGeom(residue['CG2'], residue['CB '], residue['CA '])
            methyl_list[key + ' B'] = methyl
            count_val+=1
        elif(key[0:3]=='ALA'):
            residues.append(key)
            methyl=CalcMetGeom(residue['CB '],residue['CA '],residue['C  '])
            methyl_list[key+' A']=methyl
            count_ala+=1

    #Want to return: mapping dictionaries from methyl residue name & number (EG. LEU 113 A) to numpy index, and index to name.
    #3 numpy arrays - N*3 array containing coordinates of methyl C (c_m), N*3 for orientation (v_m), and N*3*3 positions of methyl H (sites)
#     methyl = {'loc':c_m, 'orientation':v_m, 'sites':numpy.array(sites)}
    num = len(methyl_list)
    location = numpy.zeros((num, 3))
    orientation = numpy.zeros((num, 3))
    sites = numpy.zeros((num, 3, 3))
    map_key = {}
    map_index = {}
    i = 0 #Index which incremented to slot methyls into numpy arrays
    print 'count_leu = ', count_leu
    print 'count_val = ', count_val
    print 'count_ile = ', count_ile
    print 'count_ala = ', count_ala
    for key in methyl_list:
        location[i, :] = methyl_list[key]['loc']
        orientation[i, :] = methyl_list[key]['orientation']
        sites[i, :, :] = methyl_list[key]['sites']
        map_key[key] = i
        map_index[i] = key
        i+=1
        
    return location, orientation, sites, map_key, map_index, list(set(residues))

def peak_file_parser(peak_file_dir):
    file = open(peak_file_dir, 'r')
    cross,reflected={},{}
    diag_1,diag_2={},{}
    c1_shift,c2_shift={},{}
    h1_shift,h2_shift={},{}
    for line in file:
        data = line.split()
        res1 = residue_parser(data[0])
        res2 = residue_parser(data[1])
        c1,h1=data[2],data[3]
        c2,h2=data[4],data[5]
        v1,v2=data[6],data[7]
        a1,a2=data[14],data[15]
        if not(res1 in cross):
            cross[res1]={}
            reflected[res1]={}
            diag_1[res1]={}
            diag_2[res1]={}
            c1_shift[res1],c2_shift[res1]={},{}
            h1_shift[res1],h2_shift[res1]={},{}
        cross[res1][res2]=v1
        reflected[res1][res2]=v2
        diag_1[res1][res2]=a1
        diag_2[res1][res2]=a2
        c1_shift[res1][res2],c2_shift[res1][res2]=c1,c2
        h1_shift[res1][res2],h2_shift[res1][res2]=h1,h2
    map_key,map_index={},{}
    i=0
    for res1 in cross:
        map_key[res1]=i
        map_index[i]=res1
        i+=1
    
    num = len(cross)
    cross_matrix,reflected_matrix,d1_matrix,d2_matrix=numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num))
    c1_matrix,h1_matrix,c2_matrix,h2_matrix=numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num)),numpy.zeros((num,num))
    mask_matrix=numpy.ones((num,num),dtype=bool)
    pairs=[]
    for res1 in cross:
        for res2 in cross[res1]:
            pairs.append((res1,res2))
            i1,i2=map_key[res1],map_key[res2]
            cross_matrix[i1,i2]=cross[res1][res2]
            reflected_matrix[i1,i2]=reflected[res1][res2]
            d1_matrix[i1,i2]=diag_1[res1][res2]
            d2_matrix[i1,i2]=diag_2[res1][res2]
            c1_matrix[i1,i2],c2_matrix[i1,i2]=c1_shift[res1][res2],c2_shift[res1][res2]
            h1_matrix[i1,i2],h2_matrix[i1,i2]=h1_shift[res1][res2],h2_shift[res1][res2]
            mask_matrix[i1,i2]=False
    
    return cross_matrix,reflected_matrix,d1_matrix,d2_matrix,c1_matrix,h1_matrix,c2_matrix,h2_matrix,map_key,map_index,pairs,mask_matrix

def nmr_masker(h1,h2,c1,c2,map_key,pairs,threshold=0.01):
    #Takes in 4 2D square matrices. Will then calculate the difference between peaks, in an expanded 4D matrix. 
    #If two peaks are too close, IE closer than the threshold level, we eliminate them from the data set
    #IE we mask them from the 2D square matrix, setting the correspond indices to True in the mask
    
    #SETUP MATRICES TO STORE DATA:
    num=h1.shape[0]
    mask=numpy.zeros((num,num),dtype=bool)
    i_m,j_m=[],[]
    for peak1 in pairs:
        for peak2 in pairs:
            i,j,k,l=map_key[peak1[0]],map_key[peak1[1]],map_key[peak2[0]],map_key[peak2[1]]
            diff=2.*numpy.sqrt(((h1[i,j]-h1[k,l])/(h1[i,j]+h1[k,l]))**2 + ((h2[i,j]-h2[k,l])/(h2[i,j]+h2[k,l]))**2 + ((c1[i,j]-c1[k,l])/(c1[i,j]+c1[k,l]))**2 + ((c2[i,j]-c2[k,l])/(c2[i,j]+c2[k,l]))**2)
            if (diff<threshold) and (peak1[0]!=peak2[0]) and (peak1[1]!=peak2[1]):
                i_m.append(i)
                i_m.append(k)
                j_m.append(j)
                j_m.append(l)
#                 mask[i,j],mask[k,l]=True,True
#                 print 'RESIDUES WHICH ARE MASKED ARE = '
#                 print peak1,': ',h1[i,j],c1[i,j],h2[i,j],c2[i,j],'DIST = ',dist[i,j]
#                 print peak2,': ',h1[k,l],c1[k,l],h2[k,l],c2[k,l],'DIST = ',dist[k,l]
#                 print ' DIFF = ',diff,'\n'
    i_m,j_m=numpy.array(i_m),numpy.array(j_m)
    ii,jj=numpy.meshgrid(i_m,j_m,indexing='ij')
    mask[ii,jj]=True
    return mask
        
def residue_parser(residue_str):
    res_letter = residue_str[0]
    integers = [int(s) for s in re.findall(r'-?\d+\.?\d*', residue_str)]
    out_str = ''
    if(res_letter=='I'):
        out_str='ILE'+' '+str(integers[0])+' A'
    elif(res_letter=='A'):
        out_str='ALA'+' '+str(integers[0])+' A'
    elif(res_letter=='L'):
        if('D1' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' A'
        elif('D2' in residue_str):
            out_str='LEU'+' '+str(integers[0])+' B'
    elif(res_letter=='V'):
        if('G1' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' A'
        elif('G2' in residue_str):
            out_str='VAL'+' '+str(integers[0])+' B'
    if(out_str==''):
        print 'BLANK AMINO ACID CODE. AMINO ACID STRING TO BE PARSED = ',residue_str
        sys.exit(100)
    return out_str

peak_file_dir='input/correlate.3'
cross,reflected,d1,d2,c1,h1,c2,h2,map_key_nmr,map_index_nmr,pairs,mask=peak_file_parser(peak_file_dir)
pdb_file_dir='input/1EZA.pdb'
location,orientation,sites,map_key_xrd,map_index_xrd,res_list=get_pdb_coord_methyl(pdb_file_dir)
i,j=numpy.arange(location.shape[0]),numpy.arange(location.shape[0])
ii,jj=numpy.meshgrid(i,j,indexing='ij')
v_mm = location[jj, :] - location[ii, :]
dist=numpy.sqrt(numpy.sum(v_mm*v_mm,axis=2))
mask1=nmr_masker(h1,h2,c1,c2,map_key_nmr,pairs,threshold=0.10)
mask1=numpy.logical_not(mask1)
mask=numpy.logical_not(mask)
print numpy.sum(mask)
mask=numpy.logical_and(mask,mask1)
print numpy.sum(mask)


#NOW CONVERT INDICES FROM XRD TO NMR:
xrd_indices,nmr_indices=[],[]
for key in map_key_nmr:
    xrd_indices.append(map_key_xrd[key])
    nmr_indices.append(map_key_nmr[key])

xrd_indices,nmr_indices=numpy.array(xrd_indices),numpy.array(nmr_indices)
ii_xrd,jj_xrd=numpy.meshgrid(xrd_indices,xrd_indices,indexing='ij')
ii_nmr,jj_nmr=numpy.meshgrid(nmr_indices,nmr_indices,indexing='ij')

print dist[ii_xrd,jj_xrd][mask[ii_nmr,jj_nmr]]
plt.scatter(h1,h2)
plt.show()
plt.scatter(h1[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]],h2[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]])
plt.scatter(0,0)
plt.show()
plt.scatter(h1,h2)
plt.scatter(h1[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]],h2[ii_nmr,jj_nmr][mask[ii_nmr,jj_nmr]])
plt.show()






IOError: [Errno 2] No such file or directory: 'input/correlate.3'

In [95]:
# I2C-H	V229G1C-H	1.456265e+01	7.087144e-01	2.141468e+01	9.861862e-01	1.018404e+06	8.764864e+05	2.826718e+00	2.432808e+00	1.497895e-01	5.029452e-03	2.870000e-01	5.406131e-03	2.894208e+08	1.226131e+08
# I2C-H	V229G2C-H	1.456265e+01	7.087144e-01	2.103401e+01	7.734738e-01	1.240780e+07	1.048280e+07	3.443954e+01	2.909645e+01	1.681909e-01	7.661186e-02	7.479700e-02	8.305457e-02	2.894208e+08	7.711390e+07


# V36G1C-H (FAR )	I31C-H	2.179535e+01	1.084129e+00	1.494332e+01	6.883888e-01	2.830200e+06	5.290599e+06	7.855606e+00	1.468478e+01	6.059499e-01	2.270071e-02	3.900000e-01	1.582297e-02	1.438121e+08	2.224656e+08
# V36G2C-H (NEAR)	I31C-H	2.344491e+01	1.254771e+00	1.494332e+01	6.883888e-01	1.523300e+07	1.995842e+07	4.228128e+01	5.539732e+01	2.685550e-01	1.067269e-01	5.580000e-01	9.239592e-02	1.221807e+08	2.224656e+08

# A12C-H	L220D1C-H (FAR )	2.369869e+01	1.254771e+00	2.496758e+01	7.734738e-01	1.312492e+06	1.121124e+06	3.643000e+00	3.111834e+00	1.572700e-01	1.306651e-02	4.770000e-01	1.409400e-02	7.235709e+07	1.198515e+08
# A12C-H	L220D2C-H (NEAR)	2.369869e+01	1.254771e+00	2.623648e+01	7.309313e-01	4.025471e+06	3.377733e+06	1.117324e+01	9.375359e+00	1.749886e-01	4.053325e-02	5.270000e-01	4.407968e-02	7.235709e+07	1.152592e+08


# V229G1C-H	(EQUI) L218D1C-H (FAR )	2.141468e+01	9.953943e-01	2.610959e+01	6.033038e-01	5.057242e+05	0.000000e+00	1.403707e+00	0.000000e+00	2.000000e+00	2.698567e-03	3.870000e-01	5.397135e-03	1.226131e+08	7.160841e+07
# V229G1C-H	(EQUI) L218D2C-H (NEAR)	2.141468e+01	9.953943e-01	2.484069e+01	8.160163e-01	4.200693e+06	4.383431e+06	1.165960e+01	1.216681e+01	4.257570e-02	4.138277e-02	1.900000e-01	4.050182e-02	1.226131e+08	8.773158e+07
# V229G2C-H	(EQUI) L218D2C-H (NEAR)	2.103401e+01	7.837972e-01	2.484069e+01	8.160163e-01	3.316999e+06	3.439091e+06	9.206784e+00	9.545666e+00	3.614268e-02	4.106966e-02	2.220300e-02	4.032748e-02	7.711390e+07	8.773158e+07

# V229G1C-H (EQUI) V212G2C-H	2.141468e+01	9.953943e-01	2.281046e+01	8.372875e-01	1.751184e+06	6.037643e+05	4.860651e+00	1.675830e+00	9.744757e-01	1.245131e-02	1.600000e-01	1.851806e-02	1.226131e+08	7.293496e+07
# V229G2C-H	(EQUI) V212G2C-H	2.103401e+01	7.837972e-01	2.281046e+01	8.372875e-01	3.891544e+06	3.818102e+06	1.080151e+01	1.059766e+01	1.905202e-02	5.140084e-02	5.220300e-02	5.189049e-02	7.711390e+07	7.293496e+07

# A12C-H	L177D1C-H (FAR )	2.369869e+01	1.254771e+00	2.598270e+01	8.798300e-01	2.071016e+06	2.130164e+06	5.748388e+00	5.912560e+00	2.815762e-02	1.931387e-02	3.690000e-01	1.904196e-02	7.235709e+07	1.634793e+08
# A12C-H	L177D2C-H (NEAR)	2.369869e+01	1.254771e+00	2.166846e+01	8.585588e-01	1.141570e+06	1.304646e+06	3.168583e+00	3.621222e+00	1.333290e-01	9.241845e-03	3.980000e-01	8.625742e-03	7.235709e+07	2.420644e+08

# L17D1C-H	V156G1C-H (CLOSE) 2.585581e+01	8.861829e-01	2.204913e+01	8.372875e-01	1.628665e+06	2.295740e+06	4.520582e+00	6.372139e+00	3.399624e-01	1.787843e-02	5.200000e-02	1.483943e-02	7.494467e+07	1.607268e+08
# L17D2C-H	V156G1C-H (FAR) 2.382558e+01	1.022697e+00	2.204913e+01	8.372875e-01	2.452588e+06	3.062685e+06	6.807492e+00	8.500899e+00	2.212390e-01	3.265595e-02	1.906970e-01	2.904356e-02	4.436710e+07	1.607268e+08
# L17D2C-H	V156G2C-H (EQUI) 2.382558e+01	1.022697e+00	1.710044e+01	8.160163e-01	1.802337e+06	2.045500e+06	5.002633e+00	5.677564e+00	1.263891e-01	2.864422e-02	1.966970e-01	2.683406e-02	4.436710e+07	1.016807e+08
# L17D1C-H	V156G2C-H (EQUI) 2.585581e+01	8.861829e-01	1.710044e+01	8.160163e-01	5.171417e+06	5.905416e+06	1.435397e+01	1.639129e+01	1.325288e-01	6.344481e-02	5.800000e-02	5.924068e-02	7.494467e+07	1.016807e+08

# A104C-H	L137D1C-H (FAR ) 1.773488e+01	1.439066e+00	2.204913e+01	1.028729e+00	7.136694e+05	8.268384e+05	1.980887e+00	2.295003e+00	1.469242e-01	9.411152e-03	4.060000e-01	8.719789e-03	7.032311e+07	9.525422e+07
# A104C-H	L137D2C-H (NEAR) 1.773488e+01	1.439066e+00	2.572892e+01	1.092542e+00	5.851962e+06	4.936195e+06	1.624292e+01	1.370108e+01	1.697726e-01	2.996338e-01	3.540000e-01	3.250686e-01	7.032311e+07	4.608449e+06

# L80D1C-H (NEAR) I133C-H	2.661715e+01	1.043174e+00	1.468954e+01	7.734738e-01	6.365470e+05	1.122331e+06	1.766823e+00	3.115182e+00	5.523793e-01	9.379386e-03	2.620000e-01	6.788897e-03	3.327036e+07	2.642439e+08
# L80D2C-H (FAR ) I133C-H	2.446003e+01	1.009046e+00	1.468954e+01	7.734738e-01	2.170062e+06	2.912834e+06	6.023302e+00	8.084967e+00	2.922633e-01	1.711501e-02	2.270000e-01	1.461396e-02	8.344558e+07	2.642439e+08



#MISSING LEU171 - This should have a cross-peak with VAL 176
# V176G1C-H	I152C-H   (Y) 2.217602e+01	5.995030e-01	1.367442e+01	8.798300e-01	6.055478e+05	8.866936e+05	1.680781e+00	2.461139e+00	3.768100e-01	4.372311e-03	2.830000e-01	3.548546e-03	1.036670e+08	2.809025e+08
# V176G1C-H	V156G2C-H () 2.217602e+01	5.995030e-01	1.710044e+01	8.160163e-01	1.115162e+06	1.271302e+06	3.095284e+00	3.528673e+00	1.308550e-01	1.162212e-02	2.260000e-01	1.086172e-02	1.036670e+08	1.016807e+08
# V176G1C-H	L158D1C-H (Y)	2.217602e+01	5.995030e-01	2.585581e+01	9.861862e-01	4.994660e+06	6.034066e+06	1.386336e+01	1.674837e+01	1.884907e-01	2.610319e-02	3.760000e-01	2.364308e-02	1.036670e+08	4.304902e+08
# V176G1C-H	L158D2C-H (Y)	2.217602e+01	5.995030e-01	2.458692e+01	9.223725e-01	1.648013e+06	7.086340e+05	4.574286e+00	1.966910e+00	7.972167e-01	8.899381e-03	3.271370e-01	1.244675e-02	1.036670e+08	1.691100e+08
# V176G1C-H	V176G1C-H (Y)	2.217602e+01	5.995030e-01	2.217602e+01	6.033038e-01	1.036670e+08	1.036670e+08	2.877418e+02	2.877418e+02	0.000000e+00	1.000000e+00	0.000000e+00	1.000000e+00	1.036670e+08	1.036670e+08
# V176G1C-H	L197D2C-H	2.217602e+01	5.995030e-01	2.293735e+01	9.649150e-01	6.741278e+05	0.000000e+00	1.871134e+00	0.000000e+00	2.000000e+00	2.958307e-03	3.664650e-01	5.916614e-03	1.036670e+08	1.252269e+08
# V176G1C-H	L199D1C-H (Y)	2.217602e+01	5.995030e-01	2.610959e+01	8.372875e-01	3.376553e+06	3.810015e+06	9.372083e+00	1.057522e+01	1.206313e-01	3.450131e-02	2.400000e-01	3.242034e-02	1.036670e+08	1.046337e+08
# V176G2C-H	V176G2C-H (Y)	2.204913e+01	9.066600e-01	2.204913e+01	9.011012e-01	6.516352e+07	6.516352e+07	1.808702e+02	1.808702e+02	0.000000e+00	1.000000e+00	0.000000e+00	1.000000e+00	6.516352e+07	6.516352e+07




In [ ]:
# I180	V227 - Equidistant: #Comment - This guy is beautiful! Check against predictions! Have 2 pairings both at 5.2A! *But* v different intensities, not explained by initiation, *but* might be explained by orientation!
# I180C-H	V227G1C-H	1.621221e+01	7.428429e-01	2.103401e+01	7.096601e-01	6.198139e+06	6.479734e+06	1.720378e+01	1.798539e+01	4.442312e-02	7.240302e-02	2.446000e-02	7.079483e-02	1.120016e+08	6.843765e+07
# I180C-H	V227G2C-H	1.621221e+01	7.428429e-01	2.192224e+01	8.160163e-01	9.503021e+06	8.708567e+06	2.637693e+01	2.417182e+01	8.724709e-02	1.574957e-01	7.800000e-02	1.643662e-01	1.120016e+08	2.984517e+07
#PREDICTIONS AND ACTUAL FROM CODE:
# DISTANCE A =  5.205776503078096 DISTANCE B =  5.163920797223752
# SIMULATED PEAKS FOR VAL 227 A/B and ILE 180 A: 
# VAL 227 A:0.02165963457927757 0.02165963457927757  0.23181523985784958 0.1952651035194475
# VAL 227 B:0.03103892419534818 0.031038924195348178 0.23181523985784958 0.1574036684595045
# ACTUAL PEAKS FOR VAL 227 A/B and ILE 180 A: 
# VAL 227 A:6479734.0 6198139.0 112001600.0 68437650.0
# VAL 227 B:8708567.0 9503021.0 112001600.0 29845170.0
    
# I133	L79 - Near/Far: Really quite far out. But LEU is on the outside of the protein, so not surprising
# I133C-H	L79D1C-H (Far )	1.468954e+01	7.837972e-01	2.344491e+01	9.011012e-01	4.262018e+06	3.648287e+06	1.182981e+01	1.012632e+01	1.551724e-01	1.352866e-02	1.190000e-01	1.457830e-02	2.642439e+08	3.234537e+08
# I133C-H	L79D2C-H (Near)	1.468954e+01	7.837972e-01	2.547514e+01	9.223725e-01	3.247333e+06	3.934682e+06	9.013416e+00	1.092125e+01	1.914085e-01	1.885490e-02	1.320000e-01	1.705040e-02	2.642439e+08	1.372712e+08
 #PREDICTIONS AND ACTUAL FROM CODE:
# DISTANCE A =  6.406262092671514 DISTANCE B =  4.297774075030004
# SIMULATED PEAKS FOR LEU 79 A/B and ILE 133 A: 
# LEU 79 A: 0.0071400086767148575 0.0071400086767148575 0.4360397320605106 0.22933113823975815
# LEU 79 B: 0.06680428252486352   0.0668042825248635    0.4360397320605106 0.13132190368160626
# ACTUAL PEAKS FOR LEU 79 A/B and ILE 133 A: 
# LEU 79 A: 3648287.0 4262018.0 264243900.0 323453700.0
# LEU 79 B: 3934682.0 3247333.0 264243900.0 137271200.0

# I89	V107 - EQUIDISTANT: - No real correlation. VAL is on the outside of the protein
# I89C-H	V107G1C-H	1.430887e+01	6.199801e-01	2.166846e+01	9.649150e-01	2.496406e+06	2.402299e+06	6.929116e+00	6.667909e+00	3.842124e-02	3.662220e-02	3.420000e-01	3.732573e-02	1.097568e+08	4.075514e+07
# I89C-H	V107G2C-H	1.430887e+01	6.199801e-01	2.382558e+01	1.028729e+00	1.051055e+06	1.226090e+06	2.917345e+00	3.403181e+00	1.537325e-01	5.636471e-03	4.105230e-01	5.203217e-03	1.097568e+08	3.717708e+08
# DISTANCE A =  5.156327181240539 DISTANCE B =  4.900438143676543
# SIMULATED PEAKS FOR VAL 107 A/B and ILE 89 A: 
# VAL 107 A: 0.022212751227076798 0.022212751227076805 0.5042430317744989 0.223698974358902
# VAL 107 B: 0.02059592128838127  0.02059592128838127  0.5042430317744989 0.22741097340476793
# ACTUAL PEAKS FOR VAL 107 A/B and ILE 89 A: 
# VAL 107 A: 2402299.0 2496406.0 109756800.0  40755140.0
# VAL 107 B: 1226090.0 1051055.0 109756800.0 371770800.0

# I180	L220 - EQUIDISTANT-ish: - No real correlation, and on the inside of the protein. Maybe optimisation would sort this out??
# I180C-H	L220D1C-H	1.621221e+01	7.428429e-01	2.496758e+01	7.734738e-01	5.839858e+06	4.706200e+06	1.620932e+01	1.306270e+01	2.149917e-01	4.551203e-02	4.100000e-02	5.040438e-02	1.120016e+08	1.198515e+08
# I180C-H	L220D2C-H	1.621221e+01	7.428429e-01	2.623648e+01	7.309313e-01	8.338637e+05	5.571168e+05	2.314502e+00	1.546354e+00	3.979164e-01	6.121263e-03	9.000000e-03	7.339139e-03	1.120016e+08	1.152592e+08
# DISTANCE A =  3.8555401696779135 DISTANCE B =  4.179070231522797
# SIMULATED PEAKS FOR LEU 220 A/B and ILE 180 A: 
# LEU 220 A: 0.06579788919143292 0.06579788919143295 0.2300746738681491 0.07983781650253953
# LEU 220 B: 0.06756337701097898 0.06756337701097898 0.2300746738681491 0.11943033428007088
# ACTUAL PEAKS FOR LEU 220 A/B and ILE 180 A: 
# LEU 220 A: 4706200.0 5839858.0 112001600.0 119851500.0
# LEU 220 B:  557116.8  833863.7 112001600.0 115259200.0

# A16	L218 - Near/far: Qualtitatively correct, over-estimate magnitude of effect. Near the outside of the protein
# DISTANCE A =  3.316512626238591 DISTANCE B =  4.926168896820327
# SIMULATED PEAKS FOR LEU 218 A/B and ALA 16 A: 
# LEU 218 A: 0.08799092281010859 0.08799092281010858 0.2920352146028284 0.08637663101109933
# LEU 218 B: 0.02048344379030888 0.02048344379030888 0.2920352146028284 0.14479345531611462
# ACTUAL PEAKS FOR LEU 218 A/B and ALA 16 A: 
# LEU 218 A: 8303925.0 8726483.0 155422100.0 71608410.0
# LEU 218 B: 4777369.0 3929026.0 155422100.0 87731580.0

# A222	L220 - Equidistant, but quite a difference in orientation? *Missing* diagonal A222-A222 peak! 
#Qualitatively correct, needs little fine-tuning. Fairly "inside" the protein
# A222C-H	L220D1C-H	1.811555e+01	1.568754e+00	2.496758e+01	7.734738e-01	4.389571e+06	3.086185e+06	1.218385e+01	8.566127e+00	3.486969e-01	3.118757e-02	7.870000e-01	3.662508e-02	1.198515e+08	1.198515e+08
# A222C-H	L220D2C-H	1.811555e+01	1.568754e+00	2.623648e+01	7.309313e-01	5.859539e+06	3.910793e+06	1.626395e+01	1.085494e+01	3.989108e-01	4.238418e-02	8.370000e-01	5.083793e-02	1.152592e+08	1.152592e+08
# DISTANCE A =  4.541031050323264 DISTANCE B =  4.581665308596865
# SIMULATED PEAKS FOR LEU 220 A/B and ALA 222 A: 
# LEU 220 A: 0.027142830295502983 0.027142830295502993 0.40725372342552524 0.07727343259723524
# LEU 220 B: 0.03561295009710892 0.0356129500971089 0.40725372342552524 0.12245524676740302
# ACTUAL PEAKS FOR LEU 220 A/B and ALA 222 A: 
# LEU 220 A: 3086185.0 4389571.0 0.0 119851500.0
# LEU 220 B: 3910793.0 5859539.0 0.0 115259200.0

# A114	L115 - Equidistant: - Qualitatively correct. Needs a little fine-tuning. On the outside of the protein
#A114C-H	L115D1C-H	1.798866e+01	1.541451e+00	2.560203e+01	8.585588e-01	7.002436e+05	0.000000e+00	1.943622e+00	0.000000e+00	2.000000e+00	1.356056e-03	6.740000e-01	2.712113e-03	6.338062e+08	1.051783e+08
#A114C-H	L115D2C-H	1.798866e+01	1.541451e+00	2.306424e+01	9.861862e-01	1.187120e+06	1.201356e+06	3.295013e+00	3.334527e+00	1.192056e-02	4.088638e-03	5.640000e-01	4.064268e-03	6.338062e+08	1.346071e+08
# DISTANCE A =  6.189262799397033 DISTANCE B =  5.932028658056197
# SIMULATED PEAKS FOR LEU 115 A/B and ALA 114 A: 
# LEU 115 A: 0.006371674423186862 0.006371674423186862 0.6450981456059657 0.20054877621021985
# LEU 115 B: 0.008937876763610249 0.008937876763610249 0.6450981456059657 0.2308049069314677
# ACTUAL PEAKS FOR LEU 115 A/B and ALA 114 A: 
# LEU 115 A:       0.0  700243.6 633806200.0 105178300.0
# LEU 115 B: 1201356.0 1187120.0 633806200.0 134607100.0

# I133	V130 - Equidistant: Qualitatively correct, but not close. Pretty internal?
#Likely due to diagonals in reality being quite different? Predict v similar
# I133C-H	V130G1C-H	1.468954e+01	7.837972e-01	2.217602e+01	9.649150e-01	1.243054e+06	1.042593e+06	3.450267e+00	2.893858e+00	1.754093e-01	5.123526e-03	1.909170e-01	5.572883e-03	2.642439e+08	1.882849e+08
# I133C-H	V130G2C-H	1.468954e+01	7.837972e-01	2.344491e+01	1.028729e+00	1.427787e+06	1.426713e+06	3.963018e+00	3.960036e+00	7.526951e-04	1.059702e-02	2.540000e-01	1.060101e-02	2.642439e+08	6.864794e+07
# DISTANCE A =  4.957580155680794 DISTANCE B =  4.471096733464844
# SIMULATED PEAKS FOR VAL 130 A/B and ILE 133 A: 
# VAL 130 A: 0.03624711365347275 0.03624711365347275 0.43256656586320674 0.15784422314100105
# VAL 130 B: 0.0690941369899337  0.0690941369899337  0.43256656586320674 0.16159520825292362
# ACTUAL PEAKS FOR VAL 130 A/B and ILE 133 A: 
# VAL 130 A: 1042593.0 1243054.0 264243900.0 188284900.0
# VAL 130 B: 1426713.0 1427787.0 264243900.0  68647940.0



# A127	L115 - Near/Far: Qualitatively correct, needs fine-tuning
#Predicted difference about half that of actual.
#Near the outside of the protein
# A127C-H	L115D1C-H (Near) 1.913067e+01	1.684791e+00	2.560203e+01	8.585588e-01	5.951599e+06	4.734223e+06	1.651948e+01	1.314048e+01	2.278489e-01	3.589900e-02	8.220000e-01	3.998878e-02	2.106031e+08	1.051783e+08
# A127C-H	L115D2C-H (Far) 1.913067e+01	1.684791e+00	2.306424e+01	9.861862e-01	8.450931e+05	6.806885e+05	2.345671e+00	1.889344e+00	2.155022e-01	4.531020e-03	7.120000e-01	5.019243e-03	2.106031e+08	1.346071e+08
# DISTANCE A =  4.484735889659502 DISTANCE B =  5.750683959321709
# SIMULATED PEAKS FOR  LEU 115 A / LEU 115 B  AND ALA 127 A
# LEU 115 A :  0.030716577989388694 0.030716577989388694 0.5289292178744999 0.19601444658701994
# LEU 115 B :  0.008316932146793788 0.008316932146793788 0.5289292178744999 0.2327001788371809
# ACTUAL PEAKS FOR  LEU 115 A / LEU 115 B  AND ALA 127 A
# LEU 115 A :  4734223.0 5951599.0  210603100.0 105178300.0
# LEU 115 B :   680688.5  845093.1  210603100.0 134607100.0


# V36 I31 - Near/Far: In the right ball-park for difference
#Near outside of protein. Diagonals are predicted not v. accurately
# V36G1C-H (FAR )	I31C-H	2.179535e+01	1.084129e+00	1.494332e+01	6.883888e-01	2.830200e+06	5.290599e+06	7.855606e+00	1.468478e+01	6.059499e-01	2.270071e-02	3.900000e-01	1.582297e-02	1.438121e+08	2.224656e+08
# V36G2C-H (NEAR)	I31C-H	2.344491e+01	1.254771e+00	1.494332e+01	6.883888e-01	1.523300e+07	1.995842e+07	4.228128e+01	5.539732e+01	2.685550e-01	1.067269e-01	5.580000e-01	9.239592e-02	1.221807e+08	2.224656e+08
# DISTANCE A =  5.46355763216606 DISTANCE B =  3.888050153997504
# SIMULATED PEAKS FOR  VAL 36 A / VAL 36 B  AND ILE 31 A
# VAL 36 A :  0.01752923337650724 0.017529233376507237 0.5222843512643666 0.2858031554446768
# VAL 36 B :  0.1159864823017901  0.1159864823017901   0.5222843512643666 0.19665451457271005
# ACTUAL PEAKS FOR  VAL 36 A / VAL 36 B  AND ILE 31 A
# VAL 36 A :   2830200.0  5290599.0 222465600.0 143812100.0
# VAL 36 B :  15233000.0 19958420.0 222465600.0 122180700.0

# A12 L220 - Near/Far:
#Predicted difference much bigger than actual. ALA group is on the edge
# A12C-H	L220D1C-H (FAR )	2.369869e+01	1.254771e+00	2.496758e+01	7.734738e-01	1.312492e+06	1.121124e+06	3.643000e+00	3.111834e+00	1.572700e-01	1.306651e-02	4.770000e-01	1.409400e-02	7.235709e+07	1.198515e+08
# A12C-H	L220D2C-H (NEAR)	2.369869e+01	1.254771e+00	2.623648e+01	7.309313e-01	4.025471e+06	3.377733e+06	1.117324e+01	9.375359e+00	1.749886e-01	4.053325e-02	5.270000e-01	4.407968e-02	7.235709e+07	1.152592
# DISTANCE A =  6.5946889236718365 DISTANCE B =  4.797493512241575
# SIMULATED PEAKS FOR  LEU 220 A / LEU 220 B  AND ALA 12 A
# LEU 220 A :  0.0028589047237658296 0.00285890472376583 0.40111254950739 0.07776250573122219
# LEU 220 B :  0.034085714932798876  0.03408571493279887 0.40111254950739 0.12249136648025216
# ACTUAL PEAKS FOR  LEU 220 A / LEU 220 B  AND ALA 12 A
# LEU 220 A :  1121124.0 1312492.0 72357090.0 119851500.0
# LEU 220 B :  3377733.0 4025471.0 72357090.0 115259200.0

# A16	V212 - Near/Far: Qualitatively correct. Diagonals are well off! (Which probably affects the cross-peaks, so we'll need to normalize)
# A16C-H	V212G1C-H (Near) 2.052645e+01	1.288900e+00	2.293735e+01	9.223725e-01	1.153307e+07	9.586365e+06	3.201160e+01	2.660826e+01	1.843518e-01	5.893031e-02	3.620000e-01	6.436226e-02	1.554221e+08	2.065923e+08
# A16C-H	V212G2C-H (Far ) 2.052645e+01	1.288900e+00	2.281046e+01	8.372875e-01	5.913599e+05	7.724514e+05	1.641400e+00	2.144044e+00	2.655668e-01	6.404717e-03	4.510000e-01	5.554277e-03	1.554221e+08	7.293496e+07
# DISTANCE A =  3.649784788175873 DISTANCE B =  5.473381952687022
# SIMULATED PEAKS FOR  VAL 212 A / VAL 212 B  AND ALA 16 A
# VAL 212 A :  0.08633561187415353 0.08633561187415355 0.2887065350444836 0.10659713405782226
# VAL 212 B :  0.02370880303204710 0.02370880303204711 0.2887065350444836 0.14808407334055232
# ACTUAL PEAKS FOR  VAL 212 A / VAL 212 B  AND ALA 16 A
# VAL 212 A :  9586365.0 11533070.0 155422100.0 206592300.0
# VAL 212 B :   772451.4   591359.9 155422100.0  72934960.0

# I141	L137 - Near/Far: Trend is reversed to what might be expected - assignment *may* be wrong. OR diagonals need accounting for?
# I141C-H	L137D1C-H (Far ) 1.468954e+01	1.022697e+00	2.204913e+01	1.028729e+00	6.192662e+06	5.647716e+06	1.718858e+01	1.567600e+01	9.204882e-02	4.791686e-02	1.400000e-02	5.012221e-02	1.602545e+08	9.525422e+07
# I141C-H	L137D2C-H (Near) 1.468954e+01	1.022697e+00	2.572892e+01	1.092542e+00	3.666813e+06	2.227779e+06	1.017774e+01	6.183504e+00	4.882558e-01	1.084529e-01	6.600000e-02	1.349293e-01	1.602545e+08	4.608449e+06
# DISTANCE A =  4.810721775368017 DISTANCE B =  4.285826991375177
# SIMULATED PEAKS FOR  LEU 137 A / LEU 137 B  AND ILE 141 A
# LEU 137 A :  0.016366954113636256 0.016366954113636256 0.3194943459768862 0.17522180153348998
# LEU 137 B :  0.047916099484097134 0.047916099484097134 0.3194943459768862 0.14929989488350617
# ACTUAL PEAKS FOR  LEU 137 A / LEU 137 B  AND ILE 141 A
# LEU 137 A :  5647716.0 6192662.0 160254500.0 95254220.0
# LEU 137 B :  2227779.0 3666813.0 160254500.0  4608449.0

# A104	L137 - Near/Far: Qualitatively correct. Diagonal for L137D2C-H seems v off??
# A104C-H	L137D1C-H (Far ) 1.773488e+01	1.439066e+00	2.204913e+01	1.028729e+00	7.136694e+05	8.268384e+05	1.980887e+00	2.295003e+00	1.469242e-01	9.411152e-03	4.060000e-01	8.719789e-03	7.032311e+07	9.525422e+07
# A104C-H	L137D2C-H (Near) 1.773488e+01	1.439066e+00	2.572892e+01	1.092542e+00	5.851962e+06	4.936195e+06	1.624292e+01	1.370108e+01	1.697726e-01	2.996338e-01	3.540000e-01	3.250686e-01	7.032311e+07	4.608449e+06
# DISTANCE A =  6.8103388315119835 DISTANCE B =  4.476230333662467
# SIMULATED PEAKS FOR  LEU 137 A / LEU 137 B  AND ALA 104 A
# LEU 137 A :  0.00464654932682191  0.00464654932682191  0.3578153232861909 0.17522180153348998
# LEU 137 B :  0.059773504884201904 0.059773504884201925 0.3578153232861909 0.14929989488350617
# ACTUAL PEAKS FOR  LEU 137 A / LEU 137 B  AND ALA 104 A
# LEU 137 A :   826838.4  713669.4 70323110.0 95254220.0
# LEU 137 B :  4936195.0 5851962.0 70323110.0  4608449.0
    
# I133	L80 - EQUIDISTANT: #Comment - Whilst intensities of cross-peaks vary by factor of 3, this is due to *diagonal* peak intensity varying in same manner
# I133C-H	L80D1C-H	1.468954e+01	7.837972e-01	2.661715e+01	1.050000e+00	1.122331e+06	6.365470e+05	3.115182e+00	1.766823e+00	5.523793e-01	9.379386e-03	2.620000e-01	1.196988e-02	2.642439e+08	3.327036e+07
# I133C-H	L80D2C-H	1.468954e+01	7.837972e-01	2.446003e+01	1.007457e+00	2.912834e+06	2.170062e+06	8.084967e+00	6.023302e+00	2.922633e-01	1.711501e-02	2.270000e-01	1.961605e-02	2.642439e+08	8.344558e+07
# DISTANCE A =  7.15770312320929 DISTANCE B =  7.2104438143570615
# SIMULATED PEAKS FOR  LEU 80 A / LEU 80 B  AND ILE 133 A
# LEU 80 A :  0.0033110338723771567 0.0033110338723771567 0.43412403100829833 0.2179547031979381
# LEU 80 B :  0.0033427620129120018 0.0033427620129120018 0.43412403100829833 0.18128035463471912
# ACTUAL PEAKS FOR  LEU 80 A / LEU 80 B  AND ILE 133 A
# LEU 80 A :   636547.0 1122331.0 264243900.0 33270360.0
# LEU 80 B :  2170062.0 2912834.0 264243900.0 83445580.0


# I219	V235 - Near/Far: In the right direction. Diagonal peak miss-assigned? On the outside of the protein 
# I219C-H	V235G1C-H (Near) 1.494332e+01	5.790258e-01	2.116090e+01	5.182189e-01	1.036050e+07	7.090395e+06	2.875697e+01	1.968036e+01	3.747776e-01	6.624214e-02	5.300000e-02	7.865517e-02	1.566903e+08	1.107298e+08
# I219C-H	V235G2C-H (Far ) 1.494332e+01	5.790258e-01	2.306424e+01	1.050000e+00	6.500529e+05	4.756882e+05	1.804311e+00	1.320337e+00	3.097777e-01	3.101018e-03	4.680000e-01	3.581331e-03	1.566903e+08	2.102648e+08
# SIMULATED PEAKS FOR  VAL 235 A / VAL 235 B  AND ILE 219 A
# DISTANCE A =  5.464745373757135 DISTANCE B =  7.286455448295833
# VAL 235 A :  0.026878031865602926 0.026878031865602912 0.6027375276887755 0.3107601525400522
# VAL 235 B :  0.00323156421924491  0.00323156421924491  0.6027375276887755 0.3396896200674616
# ACTUAL PEAKS FOR  VAL 235 A / VAL 235 B  AND ILE 219 A
# VAL 235 A :  7090395.0 10360500.0 156690300.0 110729800.0
# VAL 235 B :   475688.2   650052.9 156690300.0 210264800.0

# I24	L138 - Near/Far: Prediction is pretty good, right ballpark.
# I24C-H	L138D1C-H (Near) 1.405509e+01	9.680914e-01	2.560203e+01	1.092542e+00	7.751588e+06	4.803168e+06	2.151559e+01	1.333185e+01	4.696897e-01	2.593678e-02	1.190000e-01	3.202789e-02	3.262101e+08	1.795673e+08
# I24C-H	L138D2C-H (Far ) 1.405509e+01	9.680914e-01	2.458692e+01	1.092542e+00	1.139144e+06	5.945201e+05	3.161848e+00	1.650172e+00	6.282920e-01	4.053399e-03	1.250000e-01	5.326758e-03	3.262101e+08	1.401953e+08
# SIMULATED PEAKS FOR  LEU 138 A / LEU 138 B  AND ILE 24 A
# DISTANCE A =  4.740466327272033 DISTANCE B =  6.16202353127607
# LEU 138 A :  0.04827898137522185  0.04827898137522183  0.5651278821654437 0.2415266170886205
# LEU 138 B :  0.006590552973503949 0.006590552973503951 0.5651278821654437 0.1598703955580448
# ACTUAL PEAKS FOR  LEU 138 A / LEU 138 B  AND ILE 24 A
# LEU 138 A :  4803168.0 7751588.0 326210100.0 179567300.0
# LEU 138 B :   594520.1 1139144.0 326210100.0 140195300.0

# A12	L177 - Near/Far (CD2): #Comment - Near the edge of the protein, PDB structure may be wrong?
#Pretty far wrong. 
# A12C-H	L177D1C-H (Far ) 2.369869e+01	1.254771e+00	2.598270e+01	8.798300e-01	2.071016e+06	2.130164e+06	5.748388e+00	5.912560e+00	2.815762e-02	1.931387e-02	3.690000e-01	1.904196e-02	7.235709e+07	1.634793e+08
# A12C-H	L177D2C-H (Near) 2.369869e+01	1.254771e+00	2.166846e+01	8.585588e-01	1.141570e+06	1.304646e+06	3.168583e+00	3.621222e+00	1.333290e-01	9.241845e-03	3.980000e-01	8.625742e-03	7.235709e+07	2.420644e+08
# SIMULATED PEAKS FOR  LEU 177 A / LEU 177 B  AND ALA 12 A
# DISTANCE A =  4.981234485546731 DISTANCE B =  3.569466906976447
# LEU 177 A :  0.02356317775393192 0.02356317775393192 0.400436433987162 0.29732524646379405
# LEU 177 B :  0.14047674634439197 0.14047674634439186 0.400436433987162 0.16953776645775429
# ACTUAL PEAKS FOR  LEU 177 A / LEU 177 B  AND ALA 12 A
# LEU 177 A :  2130164.0 2071016.0 72357090.0 163479300.0
# LEU 177 B :  1304646.0 1141570.0 72357090.0 242064400.0

# I26	L138 - EQUIDISTANT: #*About* equidistant - have v similar cross-peak intensities, reflected cross-peaks are similar-ish. Diagonals either same/similar-ish
#Quite far out - predict a factor of 2.4 difference, but actually v much closer.
#Diagonals seem to account for a far bit of the difference
#ILE 26 close to the edge of the protein, so XRD might be off?
# I26C-H	L138D1C-H	1.139041e+01	5.312458e-01	2.560203e+01	1.092542e+00	8.856864e+06	6.587779e+06	2.458343e+01	1.828528e+01	2.938345e-01	3.297020e-02	5.580000e-01	3.781409e-02	3.055099e+08	1.795673e+08
# I26C-H	L138D2C-H	1.139041e+01	5.312458e-01	2.458692e+01	1.092542e+00	8.762126e+06	6.055927e+06	2.432048e+01	1.680905e+01	3.652571e-01	3.579989e-02	5.640000e-01	4.233797e-02	3.055099e+08	1.401953e+08
# SIMULATED PEAKS FOR  LEU 138 A / LEU 138 B  AND ILE 26 A
# DISTANCE A =  5.219709283092307 DISTANCE B =  5.617826448013502
# LEU 138 A :  0.029219807046646895 0.029219807046646895 0.6014279689382217 0.2415266170886205
# LEU 138 B :  0.012147358986869024 0.012147358986869028 0.6014279689382217 0.1598703955580448
# ACTUAL PEAKS FOR  LEU 138 A / LEU 138 B  AND ILE 26 A
# LEU 138 A :  6587779.0 8856864.0 305509900.0 179567300.0
# LEU 138 B :  6055927.0 8762126.0 305509900.0 140195300.0


# A127	V130 - EQUIDISTANT?
#Prediction is qualitatively correct, needs a bit of fine tuning.
# A127C-H	V130G1C-H	1.913067e+01	1.684791e+00	2.217602e+01	9.649150e-01	9.686449e+05	6.390439e+05	2.688606e+00	1.773754e+00	4.100308e-01	4.036749e-03	7.130830e-01	4.864345e-03	2.106031e+08	1.882849e+08
# A127C-H	V130G2C-H	1.913067e+01	1.684791e+00	2.344491e+01	1.028729e+00	9.776854e+05	8.089155e+05	2.713699e+00	2.245255e+00	1.889285e-01	7.429359e-03	6.500000e-01	8.131168e-03	2.106031e+08	6.864794e+07
# SIMULATED PEAKS FOR  VAL 130 A / VAL 130 B  AND ALA 127 A
# DISTANCE A =  6.843774543334986 DISTANCE B =  6.371106732742751
# VAL 130 A :  0.00482941195398464   0.004829411953984639  0.528376837382743 0.17224073437737356
# VAL 130 B :  0.0068711306007185925 0.0068711306007185925 0.528376837382743 0.1460450790475047
# ACTUAL PEAKS FOR  VAL 130 A / VAL 130 B  AND ALA 127 A
# VAL 130 A :  639043.9 968644.9 210603100.0 188284900.0
# VAL 130 B :  808915.5 977685.4 210603100.0  68647940.0

# L44	I90 - Near/Far: #Comment - CD1 is closer, but not by much. Quite a long range guy
#Diagonals here confuse matters...Quite far out, especially accounting for diagonals.
# I90C-H	L44D1C-H (Near)	1.392820e+01	1.104606e+00	2.179535e+01	6.671176e-01	7.615102e+06	7.076904e+06	2.113675e+01	1.964291e+01	7.326408e-02	3.461742e-02	4.380000e-01	3.588553e-02	2.714899e+08	1.658666e+08
# I90C-H	L44D2C-H (Far )	1.392820e+01	1.104606e+00	2.572892e+01	9.223725e-01	4.290382e+06	3.403049e+06	1.190854e+01	9.445627e+00	2.306728e-01	3.728605e-02	1.800000e-01	4.158649e-02	2.714899e+08	3.920431e+07
# SIMULATED PEAKS FOR  LEU 44 A / LEU 44 B  AND ILE 90 A
# DISTANCE A =  5.995545096152643 DISTANCE B =  7.571003500197315
# LEU 44 A :  0.018354812729663765  0.018354812729663772  0.6094257338270073 0.3135508939039871
# LEU 44 B :  0.0034997693073592906 0.0034997693073592906 0.6094257338270073 0.31590398873527564
# ACTUAL PEAKS FOR  LEU 44 A / LEU 44 B  AND ILE 90 A
# LEU 44 A :  7076904.0 7615102.0 271489900.0 165866600.0
# LEU 44 B :  3403049.0 4290382.0 271489900.0  39204310.0

